In [7]:
######################
## import libraries ##
######################
import pandas as pd
import numpy as np    
np.random.seed(108156571)
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [8]:
def simulate_dataset(n=100000, C1=1):
    
    ## A: dichotomous
    ## C1: continuous (with squared term) -> only effect modification
    ## C2: discrete -> only confouding
        
    A_split = 0.4
    C2_split = 0.35

    df_A1 = pd.DataFrame()
    df_A1['A'] = np.ones(int(n*A_split))
    df_A1['C2'] = np.random.uniform(0, 1, df_A1.shape[0])
    df_A1.loc[df_A1['C2']<C2_split, 'C2'] = 0
    df_A1.loc[df_A1['C2']>=C2_split, 'C2'] = 1
        
        
    df_A0 = pd.DataFrame()
    df_A0['A'] = np.zeros(int(n*(1-A_split)))
    df_A0['C2'] = np.random.uniform(0, 1, df_A0.shape[0])
    df_A0.loc[df_A0['C2']<(1-C2_split), 'C2'] = 0
    df_A0.loc[df_A0['C2']>=(1-C2_split), 'C2'] = 1
    
    df = pd.concat([df_A0, df_A1], axis=0).reset_index(drop=True) 
    df['C1'] = np.random.normal(2.5, 1, df.shape[0])
    df['C1_squared'] = df['C1']*df['C1']
    df['C1_sin'] = 3*np.sin(df['C1'])
    df['C1_squared_sin_interaction'] = df['C1_squared']*df['C1_sin']
    df['error'] = np.random.normal(0, 0.25, df.shape[0])     
    
    B0 = -0.23
    B1 = 1.23
    B2 = 1.56
    B3 = 1.42
    B4 = 2.31
    B5 = -0.43
    B6 = 0.52
    B7 = 1.09
    
    df['Y'] = B0 + (B1*df['A']) + (B2*df['C1']) + (B3*df['C1_squared']) + (B4*df['C1_sin']) + (B5*df['C1_squared_sin_interaction']) + (B6*df['C2']) + (B7*df['A']*df['C1']) + df['error']
    
    print("In the marginal Causal DAG, there is confounding by C2 and effect modification by C1 \n")
    print("The true outcome model is:")
    print('E[Y|A,C1,C2] = ' + str(B0) + ' + ' + str(B1) + '*A + ' + str(B2) + '*C1 + ' + str(B3) + '*C1^2 + ' + str(B4) + '*sin(C1) + ' + str(B5) + '*C1^2*sin(C1) + ' + str(B6) + '*C2 + ' + str(B7) + '*A*C1')
    print('\n')
    print('The true Mean Causal Effect Difference under intervention a=1 and a=0 within levels of C1=' + str(C1) + ' is:')
    print('E[Y(a=1)-Y(a=0)|C1=' + str(C1) + '] = ' + str(B1 + (B7*C1)))
   
    return(df)

In [9]:
## get simulated dataset
df_original = simulate_dataset(n=200000, C1=0.5)

In the marginal Causal DAG, there is confounding by C2 and effect modification by C1 

The true outcome model is:
E[Y|A,C1,C2] = -0.23 + 1.23*A + 1.56*C1 + 1.42*C1^2 + 2.31*sin(C1) + -0.43*C1^2*sin(C1) + 0.52*C2 + 1.09*A*C1


The true Mean Causal Effect Difference under intervention a=1 and a=0 within levels of C1=0.5 is:
E[Y(a=1)-Y(a=0)|C1=0.5] = 1.775


In [10]:
###############################################
## fit and return standardized outcome model ##
###############################################
def fit_standardized_outcome_model(df, model_string, C1, fit_bounds=True, print_result=True):
    df_original = df.copy()
    df_A0 = df_original.copy()
    df_A1 = df_original.copy()
    
    df_A0['A'] = 0
    df_A0['C1'] = C1
    df_A0['C1_squared'] = df_A0['C1']*df_A0['C1']
    df_A0['C1_sin'] = 3*np.sin(df_A0['C1'])
    df_A0['C1_squared_sin_interaction'] = df_A0['C1_squared']*df_A0['C1_sin']
    
    df_A1['A'] = 1
    df_A1['C1'] = C1
    df_A1['C1_squared'] = df_A1['C1']*df_A1['C1']
    df_A1['C1_sin'] = 3*np.sin(df_A1['C1'])
    df_A1['C1_squared_sin_interaction'] = df_A1['C1_squared']*df_A1['C1_sin']

    outcome_model = smf.ols(formula=model_string, data=df).fit()
    df_A0['Y_fit'] = outcome_model.predict(df_A0)
    df_A1['Y_fit'] = outcome_model.predict(df_A1)
    mean_unexposed = df_A0['Y_fit'].mean()
    mean_exposed = df_A1['Y_fit'].mean()
    effect_difference = mean_exposed - mean_unexposed
        
    if(fit_bounds):
        effect_difference_list = []
        for i in range(0, 100):
            df_sub = df_original.sample(n=df_original.shape[0], replace=True)
            df_A0_sub = df_sub.copy()
            df_A1_sub = df_sub.copy()
            
            df_A0_sub['A'] = 0
            df_A0_sub['C1'] = C1
            df_A0_sub['C1_squared'] = df_A0_sub['C1']*df_A0_sub['C1']
            df_A0_sub['C1_sin'] = 3*np.sin(df_A0_sub['C1'])
            df_A0_sub['C1_squared_sin_interaction'] = df_A0_sub['C1_squared']*df_A0_sub['C1_sin']
            
            df_A1_sub['A'] = 1
            df_A1_sub['C1'] = C1
            df_A1_sub['C1_squared'] = df_A1_sub['C1']*df_A1_sub['C1']
            df_A1_sub['C1_sin'] = 3*np.sin(df_A1_sub['C1'])
            df_A1_sub['C1_squared_sin_interaction'] = df_A1_sub['C1_squared']*df_A1_sub['C1_sin']
            
            OM_sub = smf.ols(formula=model_string, data=df_sub).fit()
            df_A0_sub['Y_fit'] = OM_sub.predict(df_A0_sub)
            df_A1_sub['Y_fit'] = OM_sub.predict(df_A1_sub)
            mean_unexposed_sub = df_A0_sub['Y_fit'].mean()
            mean_exposed_sub = df_A1_sub['Y_fit'].mean()
            effect_difference_sub = mean_exposed_sub - mean_unexposed_sub
            effect_difference_list.append(effect_difference_sub)
            del df_sub, df_A0_sub, df_A1_sub, OM_sub, mean_unexposed_sub, mean_exposed_sub, effect_difference_sub
   
        variance = np.var(effect_difference_list)
        standard_error = np.sqrt(variance)
        bound_size = 1.96*standard_error
        lower_bound = effect_difference - bound_size
        upper_bound = effect_difference + bound_size
    
    if(print_result):
        print(outcome_model.summary())
        print("The standardized mean in those with A=0 and C1=" + str(C1) +" is: " + str(mean_unexposed))
        print("The standardized mean in those with A=1 and C1=" + str(C1) +" is: " + str(mean_exposed))
        if(fit_bounds):
            print("The standardized effect difference with C1=" + str(C1) +" is " + str(effect_difference) + " with 95% CI (" + str(lower_bound) + ' , ' + str(upper_bound) + ')')
        else:
            print("The standardized effect difference with C1=" + str(C1) + " is " + str(effect_difference))
    
    return(df, outcome_model)

In [11]:
## Standardization via the g-formula with correctly specified Outcome model
_, OM_CS = fit_standardized_outcome_model(df_original.copy(), model_string='Y ~ A + C1 + C1_squared + C1_sin + C1_squared_sin_interaction + C2 + A*C1', C1=0.5)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.241e+07
Date:                Tue, 15 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:30:44   Log-Likelihood:                -6440.0
No. Observations:              200000   AIC:                         1.290e+04
Df Residuals:                  199992   BIC:                         1.298e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [13]:
## Standardization via the g-formula with incorrectly specified Outcome model
_, OM_IS = fit_standardized_outcome_model(df_original.copy(), model_string='Y ~ A + C1 + C2 + A*C1', C1=0.5)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                 1.787e+05
Date:                Tue, 15 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:31:45   Log-Likelihood:            -6.3824e+05
No. Observations:              200000   AIC:                         1.276e+06
Df Residuals:                  199995   BIC:                         1.277e+06
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.3879      0.047   -201.192      0.0

In [14]:
##############################################
## fit and return Marginal Strucutral Model ##
##############################################
def fit_MSM_model(df, model_string, MSM_string, C1, fit_bounds=True, print_result=True):
    PS_model = smf.glm(model_string, data=df, family=sm.families.Binomial()).fit()
    df['Propensity_Score'] = PS_model.predict(df)
    Marg_Prob_Intervention = df.loc[df['A']==1, :].reset_index(drop=True).shape[0] / df.shape[0]
    df['Unstabalized_Weights'] = None
    df.loc[df['A']==1, 'Unstabalized_Weights'] = 1 / (df.loc[df['A']==1, 'Propensity_Score'])
    df.loc[df['A']==0, 'Unstabalized_Weights'] = 1 / (1 - df.loc[df['A']==0, 'Propensity_Score'])
    df['Stabalized_Weights'] = None
    df.loc[df['A']==1, 'Stabalized_Weights'] = (Marg_Prob_Intervention) / (df.loc[df['A']==1, 'Propensity_Score'])
    df.loc[df['A']==0, 'Stabalized_Weights'] = (1 - Marg_Prob_Intervention) / (1 - df.loc[df['A']==0, 'Propensity_Score'])
    MSM = smf.wls(MSM_string, data=df, weights=np.array(df['Stabalized_Weights'], dtype=np.float64)).fit()
    #print(MSM.summary())
    
    df_A0 = pd.DataFrame()
    df_A0['A'] = [0]
    df_A0['C1'] = C1
    df_A0['C1_squared'] = df_A0['C1']*df_A0['C1']
    df_A0['C1_sin'] = 3*np.sin(df_A0['C1'])
    df_A0['C1_squared_sin_interaction'] = df_A0['C1_squared']*df_A0['C1_sin']
    df_A1 = df_A0.copy()
    df_A1['A'] = 1
    mean_unexposed = MSM.predict(df_A0)[0]
    mean_exposed = MSM.predict(df_A1)[0]
    effect_difference = mean_exposed - mean_unexposed
    
    if(fit_bounds):
        effect_difference_list = []
        for i in range(0, 100):
            df_sub = df.sample(n=df.shape[0], replace=True)
            PS_model_sub = smf.glm(model_string, data=df_sub, family=sm.families.Binomial()).fit()
            df_sub['Propensity_Score'] = PS_model_sub.predict(df_sub)
            Marg_Prob_Intervention_sub = df_sub.loc[df_sub['A']==1, :].reset_index(drop=True).shape[0] / df_sub.shape[0]
            df_sub['Unstabalized_Weights'] = None
            df_sub.loc[df_sub['A']==1, 'Unstabalized_Weights'] = 1 / (df_sub.loc[df_sub['A']==1, 'Propensity_Score'])
            df_sub.loc[df_sub['A']==0, 'Unstabalized_Weights'] = 1 / (1 - df_sub.loc[df_sub['A']==0, 'Propensity_Score'])
            df_sub['Stabalized_Weights'] = None
            df_sub.loc[df_sub['A']==1, 'Stabalized_Weights'] = (Marg_Prob_Intervention_sub) / (df_sub.loc[df_sub['A']==1, 'Propensity_Score'])
            df_sub.loc[df_sub['A']==0, 'Stabalized_Weights'] = (1 - Marg_Prob_Intervention_sub) / (1 - df_sub.loc[df_sub['A']==0, 'Propensity_Score'])
            MSM_sub = smf.wls(MSM_string, data=df_sub, weights=np.array(df_sub['Stabalized_Weights'], dtype=np.float64)).fit()
            
            df_A0_sub = pd.DataFrame()
            df_A0_sub['A'] = [0]
            df_A0_sub['C1'] = C1
            df_A0_sub['C1_squared'] = df_A0_sub['C1']*df_A0_sub['C1']
            df_A0_sub['C1_sin'] = 3*np.sin(df_A0_sub['C1'])
            df_A0_sub['C1_squared_sin_interaction'] = df_A0_sub['C1_squared']*df_A0_sub['C1_sin']
            df_A1_sub = df_A0_sub.copy()
            df_A1_sub['A'] = 1
            mean_unexposed_sub = MSM_sub.predict(df_A0_sub)[0]
            mean_exposed_sub = MSM_sub.predict(df_A1_sub)[0]
            effect_difference_sub = mean_exposed_sub - mean_unexposed_sub
            
            effect_difference_list.append(effect_difference_sub)
            
            del df_sub, PS_model_sub, Marg_Prob_Intervention_sub, MSM_sub, df_A0_sub, df_A1_sub, mean_unexposed_sub, mean_exposed_sub, effect_difference_sub
    
        variance = np.var(effect_difference_list)
        standard_error = np.sqrt(variance)
        bound_size = 1.96*standard_error
        lower_bound = effect_difference - bound_size
        upper_bound = effect_difference + bound_size

    if(print_result):
        print(MSM.summary())
        print("The MSM mean in those with A=0 and C1=" + str(C1) + " is:" + str(mean_unexposed))
        print("The MSM mean in those with A=1 and C1=" + str(C1) + " is:" + str(mean_exposed))
        
        if(fit_bounds):
            print("The MSM effect difference with C1=" + str(C1) +" is " + str(effect_difference) + " with 95% CI (" + str(lower_bound) + ' , ' + str(upper_bound) + ')')
        else:
            print("The MSM effect difference with C1=" + str(C1) + " is " + str(effect_difference))
        
    return(df, MSM)

In [15]:
## Inverse Probability Weighting via Marginal Structural Modeling, 
## with correctly specified Intervention model and Marginal Structural Model
df_MSM_CS, MSM_CS = fit_MSM_model(df_original.copy(), model_string='A ~ C2', MSM_string='Y ~ A + C1 + C1_squared + C1_sin + C1_squared_sin_interaction + A*C1', C1=0.5)

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.999
Model:                            WLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 4.069e+07
Date:                Tue, 15 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:34:39   Log-Likelihood:                -83887.
No. Observations:              200000   AIC:                         1.678e+05
Df Residuals:                  199993   BIC:                         1.679e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [16]:
## Inverse Probability Weighting via Marginal Structural Modeling, 
## with incorrectly specified Intervention model and Marginal Structural Model
df_MSM_CS, MSM_CS = fit_MSM_model(df_original.copy(), model_string='A ~ C2', MSM_string='Y ~ A + C1 + A*C1', C1=0.5)

                            WLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.781
Model:                            WLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                 2.376e+05
Date:                Tue, 15 Dec 2020   Prob (F-statistic):               0.00
Time:                        12:38:48   Log-Likelihood:            -6.4287e+05
No. Observations:              200000   AIC:                         1.286e+06
Df Residuals:                  199996   BIC:                         1.286e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.1624      0.046   -200.380      0.0

In [34]:
############################################
## fit and return Strucutral Nested Model ##
############################################

def recover_psi_estimates(df_original, psi_1_bounds, psi_2_bounds, n=21):
    psi_1_list = np.linspace(psi_1_bounds[0], psi_1_bounds[1], n)
    psi_2_list = np.linspace(psi_2_bounds[0], psi_2_bounds[1], n)
    df_results = pd.DataFrame()
    df_results['psi_1'] = (n**2)*[None]
    df_results['psi_2'] = None
    df_results['coeff_Y_a0'] = None
    df_results['coeff_Y_a0_C1'] = None
    df_results['mean_error'] = None
    i=0
    for psi_1 in psi_1_list:
        for psi_2 in psi_2_list:
            df = df_original.copy()
            df['Y_a0'] = df['Y'] - psi_1*df['A'] - psi_2*df['A']*df['C1']
            df['Y_a0_C1'] = df['Y_a0']*df['C1']
            PS_model = smf.glm('A ~ C2 + Y_a0 + Y_a0_C1', data=df, family=sm.families.Binomial()).fit()
            df_results.loc[i, 'psi_1'] = psi_1
            df_results.loc[i, 'psi_2'] = psi_2
            df_results.loc[i, 'coeff_Y_a0'] = PS_model.params[2]
            df_results.loc[i, 'coeff_Y_a0_C1'] = PS_model.params[3]
            df_results.loc[i, 'mean_error'] = np.mean([abs(PS_model.params[2]), abs(PS_model.params[3])])
            del df, PS_model
            i = i+1
    df_results = df_results.loc[df_results['mean_error']==min(df_results['mean_error']), :].reset_index(drop=True)
    psi_1 = df_results.loc[0, 'psi_1']
    psi_2 = df_results.loc[0, 'psi_2']    
    return(psi_1, psi_2)

def recover_SNM_with_bounds(df_original, psi_1_bounds, psi_2_bounds, C1=1, n=21, m=10):
    psi_1, psi_2 = recover_psi_estimates(df_original, psi_1_bounds=psi_1_bounds, psi_2_bounds=psi_2_bounds, n=n)
    ED = psi_1 + (psi_2*C1)
    print("Recovered point-estimates of location parameters.")
    
    psi_1_Boostrap = []
    psi_2_Boostrap = []
    ED_Boostrap = []
    for i in range(0, m):
        df_BS = df_original.sample(n=df_original.shape[0], replace=True)
        psi_1_BS, psi_2_BS = recover_psi_estimates(df_BS, psi_1_bounds=psi_1_bounds, psi_2_bounds=psi_2_bounds, n=n)
        ED_BS = psi_1_BS + (psi_2_BS*C1)
        psi_1_Boostrap.append(psi_1_BS)
        psi_2_Boostrap.append(psi_2_BS)
        ED_Boostrap.append(ED_BS)
        del df_BS, psi_1_BS, psi_2_BS, ED_BS
        print("Completed " + str(i+1) + " Bootstrap Iteration.")
    
    psi_1_se = np.sqrt(np.var(psi_1_Boostrap))
    psi_2_se = np.sqrt(np.var(psi_2_Boostrap))
    ED_se = np.sqrt(np.var(ED_Boostrap))
    
    psi_1 = np.mean(psi_1_Boostrap)
    psi_2 = np.mean(psi_2_Boostrap)
    ED = np.mean(ED_Boostrap)
        
    psi_1_lb = psi_1 - (1.96*psi_1_se)  
    psi_1_ub = psi_1 + (1.96*psi_1_se) 
    psi_2_lb = psi_2 - (1.96*psi_2_se)  
    psi_2_ub = psi_2 + (1.96*psi_2_se)
    ED_lb = ED - (1.96*ED_se)  
    ED_ub = ED + (1.96*ED_se)
    
    print('\n')
    print('The estimate for theta_1 is ' + str(psi_1) + ' with 95% CI (' + str(psi_1_lb) + ' , ' + str(psi_1_ub) + ')')
    print('\n')
    print('The estimate for theta_6 is ' + str(psi_2) + ' with 95% CI (' + str(psi_2_lb) + ' , ' + str(psi_2_ub) + ')')
    print('\n')
    print('The estimated Mean Causal Effect Difference under intervention a=1 and a=0 within levels of C1=' + str(C1) + ' is ' + str(ED) + ' with 95% CI (' + str(ED_lb) + ' , ' + str(ED_ub) + ')')

    return(psi_1, [psi_1_lb, psi_1_ub], psi_2, [psi_2_lb, psi_2_ub])

In [35]:
## Fit Structural Nested Model
theta_1, theta_1_95CI, theta_6, theta_6_95CI = recover_SNM_with_bounds(df_original, psi_1_bounds=[1.2, 1.3], psi_2_bounds=[1, 1.1], C1=0.5, n=6, m=200)

Recovered point-estimates of location parameters.
Completed 1 Bootstrap Iteration.
Completed 2 Bootstrap Iteration.
Completed 3 Bootstrap Iteration.
Completed 4 Bootstrap Iteration.
Completed 5 Bootstrap Iteration.
Completed 6 Bootstrap Iteration.
Completed 7 Bootstrap Iteration.
Completed 8 Bootstrap Iteration.
Completed 9 Bootstrap Iteration.
Completed 10 Bootstrap Iteration.
Completed 11 Bootstrap Iteration.
Completed 12 Bootstrap Iteration.
Completed 13 Bootstrap Iteration.
Completed 14 Bootstrap Iteration.
Completed 15 Bootstrap Iteration.
Completed 16 Bootstrap Iteration.
Completed 17 Bootstrap Iteration.
Completed 18 Bootstrap Iteration.
Completed 19 Bootstrap Iteration.
Completed 20 Bootstrap Iteration.
Completed 21 Bootstrap Iteration.
Completed 22 Bootstrap Iteration.
Completed 23 Bootstrap Iteration.
Completed 24 Bootstrap Iteration.
Completed 25 Bootstrap Iteration.
Completed 26 Bootstrap Iteration.
Completed 27 Bootstrap Iteration.
Completed 28 Bootstrap Iteration.
Complet